## 1. check corpus

In [9]:
import pickle
import torch

# ======= 手动改这里：你的 corpus.pkl 路径 =======
CORPUS_PATH = "/Users/kiancai/STA24/CWD/STAi/MiCoGPT/data/try2_withCC/abundance_all_90338.pkl"

# 想重点查看的样本 index（可以按需要改）
CHECK_INDICES = [0, 1, 2]  # 例如前几个样本
WINDOW = 5  # 看 eos 前后各多少个 token


def main():
    # 1. 读入 corpus 对象
    with open(CORPUS_PATH, "rb") as f:
        corpus = pickle.load(f)

    tokens = corpus.tokens  # shape: (num_samples, max_len)
    pad_id = corpus.tokenizer.pad_token_id
    eos_id = getattr(corpus.tokenizer, "eos_token_id", None)
    bos_id = getattr(corpus.tokenizer, "bos_token_id", None)

    print(f"Loaded corpus from: {CORPUS_PATH}")
    print(f"tokens.shape = {tokens.shape}")
    print(f"pad_token_id = {pad_id}")
    print(f"bos_token_id = {bos_id}")
    print(f"eos_token_id = {eos_id}")

    if eos_id is None:
        print("⚠ tokenizer 没有 eos_token_id 属性，先确认 tokenizer 里是怎么定义 eos 的。")
        return

    # 2. 计算每个样本的“有效长度”（非 pad token 的数量）
    non_pad_mask = (tokens != pad_id)
    lengths = non_pad_mask.sum(dim=1)  # (num_samples,)
    max_len_val = int(lengths.max())
    min_len_val = int(lengths.min())
    idx_max = int(lengths.argmax())
    idx_min = int(lengths.argmin())

    print("\n=== 有效长度统计（非 pad 数量）===")
    print(f"样本数: {tokens.size(0)}")
    print(f"最短长度: {min_len_val}  (样本 index = {idx_min})")
    print(f"最长长度: {max_len_val}  (样本 index = {idx_max})")

    # 3. 统计 eos 的情况
    num_no_eos = 0
    num_eos_at_last_nonpad = 0
    num_eos_before_last_nonpad = 0

    eos_positions = []  # 保存每个样本的 eos 位置（如果存在）

    for i in range(tokens.size(0)):
        row = tokens[i]
        # 找到所有 eos 的位置（通常应该只有一个）
        eos_idx = (row == eos_id).nonzero(as_tuple=True)[0]
        if eos_idx.numel() == 0:
            num_no_eos += 1
            eos_positions.append(None)
            continue

        # 如果有多个 eos，就取最后一个（通常不会这样）
        eos_pos = int(eos_idx[-1])
        eos_positions.append(eos_pos)

        last_nonpad_pos = int(lengths[i].item() - 1)  # 最后一个非 pad 的下标

        if eos_pos == last_nonpad_pos:
            num_eos_at_last_nonpad += 1
        elif eos_pos < last_nonpad_pos:
            num_eos_before_last_nonpad += 1
        else:
            # eos 在 pad 区域之后（理论上不太应该发生），也可以打印出来看
            pass

    print("\n=== <eos> 位置统计 ===")
    print(f"没有 eos 的样本数: {num_no_eos}")
    print(f"eos 正好在最后一个非 pad 位置的样本数: {num_eos_at_last_nonpad}")
    print(f"eos 出现在最后一个非 pad 之前的样本数: {num_eos_before_last_nonpad}")

    # 4. 打印几个指定样本，查看 eos 前后 token id / 文本
    def show_sample(idx: int):
        if idx < 0 or idx >= tokens.size(0):
            print(f"\n样本 index {idx} 越界，跳过")
            return

        row = tokens[idx]
        length = int(lengths[idx].item())
        eos_pos = eos_positions[idx]

        print("\n" + "=" * 60)
        print(f"样本 index = {idx}")
        print(f"有效长度（非 pad） = {length}")
        print(f"eos 位置 = {eos_pos}")

        # 打印整行 token id 前若干个（防止太长）
        print("前 30 个 token id:")
        print(row[:30].tolist())

        if eos_pos is not None:
            start = max(0, eos_pos - WINDOW)
            end = min(row.size(0), eos_pos + WINDOW + 1)
            print(f"\n[eos] 附近 token id （从 {start} 到 {end - 1}）:")
            print(row[start:end].tolist())

            # 如果 tokenizer 支持 decode，可以尝试解成文本看看
            if hasattr(corpus.tokenizer, "decode"):
                try:
                    # 这里只看不含 pad 的一段
                    ids_segment = row[start:end].tolist()
                    text = corpus.tokenizer.decode(ids_segment)
                    print("\n[eos] 附近 decode 文本:")
                    print(text)
                except Exception as e:
                    print(f"\ndecode 失败: {e}")

    # 把几个感兴趣的样本（手动指定 + 最短 + 最长）都看一下
    all_to_check = set(CHECK_INDICES + [idx_min, idx_max])
    for idx in sorted(all_to_check):
        show_sample(idx)


if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: '/Users/kiancai/STA24/CWD/STAi/MiCoGPT/data/try2_withCC/abundance_all_90338.pkl'